***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin

In [1]:
from gensim.models.keyedvectors import KeyedVectors

wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [2]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [3]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

In [4]:
# method most_simmilar
def most_similar(words:list, embeddings, inclusion=False, print_negative=True):
    '''
    words: список слов, наличие которых проверяем в векторных представленияx.
    embeddings: массив векторных представлений.
    inclusion: проверяет входит ли inclusion в Топ-5 близких слов к словам words.
    print_negative: выводить или нет на печать отсутствующие слова.
    Если да, указывает место в рейтинге вхождений.  
    '''
    for word in words:
        if word in embeddings:
            print(f'Слово {word} присутствует в векторных представлениях')
            similar = dict(embeddings.most_similar(word))
            if inclusion:
                try:
                    position = sorted(similar.values(), reverse=True).index(similar[inclusion])+1
                    if position <= 5:
                        print(f'Слово {inclusion} входит в Топ-5 близких слов к слову {word} под номером {position}')
                except KeyError:
                    print(f'Слово {inclusion} не входит в Топ-5 близких слов к слову {word}')    
        else:
            if print_negative:
                print(f'Слово {word} отсутствует в векторных представлениях')

Проверим, входит ли слово `cat` в топ-5 близких слов к слову `dog`?

In [5]:
most_similar(['dog'], wv_embeddings, 'cat')

Слово dog присутствует в векторных представлениях
Слово cat не входит в Топ-5 близких слов к слову dog


Нет, не входит.

Проверим ещё, может слово `cats` входит?

In [6]:
most_similar(['dog'], wv_embeddings, 'cats')

Слово dog присутствует в векторных представлениях
Слово cats входит в Топ-5 близких слов к слову dog под номером 4


Да, есть вхождение.

Попробуем поменять `cat` и `dog` местами:

In [7]:
most_similar(['cat'], wv_embeddings, 'dog')

Слово cat присутствует в векторных представлениях
Слово dog входит в Топ-5 близких слов к слову cat под номером 2


Проверим, приведены ли слова в предобученных эмбеддингах к `lower`:

In [8]:
most_similar(['Python', 'python', 'Apple', 'apple', 'Internet', 'internet', 'Microsoft', 'microsoft'], wv_embeddings)

Слово Python отсутствует в векторных представлениях
Слово python присутствует в векторных представлениях
Слово Apple отсутствует в векторных представлениях
Слово apple присутствует в векторных представлениях
Слово Internet отсутствует в векторных представлениях
Слово internet присутствует в векторных представлениях
Слово Microsoft отсутствует в векторных представлениях
Слово microsoft присутствует в векторных представлениях


Слово `Internet` по правилам английского языка должно быть написано с большой буквы, впрочем как и `Microsoft`. Теперь очевидно, что слова при обучении эмбеддингов были приведены к `lower`. Таким образом имеет смысл в дальнейшем приводить слова к `lower` при их поиске в векторных представлениях.

Проверим были ли нормализованы слова перед обучением эмбеддингов, для этого попробуем найти разные формы одного и того же слова в векторных представлениях:

In [9]:
most_similar(['recognised', 'recogniser', 'recognise', 'recognising'], wv_embeddings)

Слово recognised присутствует в векторных представлениях
Слово recogniser присутствует в векторных представлениях
Слово recognise присутствует в векторных представлениях
Слово recognising присутствует в векторных представлениях


В векторных представлениях присутствуют различные формы одного и того же слова, поэтому предварительная нормализация слов для создания токенов не обязательна.

Проверим наличие `Стоп Слов` в векторных представлениях:

In [10]:
from nltk.corpus import stopwords

stopWords = list(stopwords.words('english'))

In [11]:
most_similar(stopWords, wv_embeddings, print_negative=False)

Слово ours присутствует в векторных представлениях
Слово himself присутствует в векторных представлениях
Слово will присутствует в векторных представлениях
Слово just присутствует в векторных представлениях
Слово now присутствует в векторных представлениях
Слово o присутствует в векторных представлениях
Слово y присутствует в векторных представлениях
Слово ain присутствует в векторных представлениях
Слово ma присутствует в векторных представлениях
Слово mightn присутствует в векторных представлениях
Слово needn присутствует в векторных представлениях


Лишь только малая часть стоп слов присутствует в векторных представлениях, поэтому предварительная фильтрация по листу стоп слов для создания токенов не обязательна.

Проверим наличие знаков пунктуации в векторных представлениях:

In [12]:
import string

most_similar(string.punctuation, wv_embeddings, print_negative=False)

Слово # присутствует в векторных представлениях
Слово + присутствует в векторных представлениях
Слово - присутствует в векторных представлениях
Слово / присутствует в векторных представлениях
Слово \ присутствует в векторных представлениях
Слово _ присутствует в векторных представлениях


В векторных представлениях присутствуют лишь несколько знаков пунктуации.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

Применим различные типы токенайзеров:

1. Используем собственный класс токенайзера, основанный на регулярных выражения и возвращающий слова без знаков пунктуации и без применения нормализации.

In [13]:
import re

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

re_tokenizer = MyTokenizer()

2. Загрузим библиотечный токенайзер, который помимо слов токенизирует знаки пунктуации, но не применяет нормализацию.

In [14]:
from nltk.tokenize import WordPunctTokenizer

wp_tokenizer = WordPunctTokenizer()

3. Загрузим ещё один библиотечный токенайзер, который помимо слов токенизирует знаки пунктуации, но применяет нормализацию (лемматизацию)

In [15]:
import spacy

nlp = spacy.load('en_core_web_sm')

class SpacyTokenizer():  
    def __init__(self):
        pass
    def tokenize(self, text):
        spacy_results = nlp(text)
        return [token.lemma_ for token in spacy_results]

spacy_tokenizer = SpacyTokenizer()

In [16]:
# словарь используемых токенайзеров
tokenizers = {'RE_tokenizer':re_tokenizer, 'WP_tokenizer':wp_tokenizer, 'Spacy_tokenizer':spacy_tokenizer}

In [17]:
import numpy as np

In [18]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        tokenizer: токенайзер
        dim: размер любого вектора в нашем представлении
        return: векторное представление для вопроса
    """ 
    itr = 0
    tokens = tokenizer.tokenize(question.lower())  # приведём к нижнему регистру
    q_vector = np.zeros(dim, dtype='float32')
    
    for token in tokens:
        if token in embeddings:
            q_vector += embeddings[token]
            itr += 1
    
    if itr:
        return q_vector / itr  # если есть хоть один эмбеддинг, возвращаем среднее
    else:
        return q_vector        # если нет, - возвращаем нулевой вектор

Теперь у нас есть метод для создания векторного представления любого предложения.

In [19]:
string = 'I love neural networks'

for name, tokenizer in tokenizers.items():
    string_vector = question_to_vec(string, wv_embeddings, tokenizer)
    print(f'Токенайзер: {name}, результат: {string_vector[2]:.2f}')

Токенайзер: RE_tokenizer, результат: -1.29
Токенайзер: WP_tokenizer, результат: -1.29
Токенайзер: Spacy_tokenizer, результат: -1.41


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

Максимум `Hits@47 - DCG@1`?

Максимум разности двух метрик достигается при максимуме уменьшаемой метрики (`Hits@47`) и минимуме вычитаемой метрики (`DCG@1`). Из определений метрик очевидно, что максимум обоих метрик равен `1` при $rank\_q_i^{'} = 1$ для каждого из вопросов `N` вне зависимости от `K` (`K` не может быть меньше `1`) . Минимум метрик равен `0` при $rank\_q_i^{'} > K$ для каждого из вопросов `N`. Если `Hits@47 - DCG@1` взаимосвязаны и относятся к одному примеру, то максимум `Hits@47 - DCG@1` равен `1 - 0 = 1` при $1 < rank\_q_i^{'} < 48$.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

Вычислим `DCG@10`, если $rank\_q_i^{'} = 9$

In [20]:
import math

K = 10
rank_q = 9

DCG = (1 / math.log2(1 + rank_q)) * (rank_q <= K)
round(DCG, 1)

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [21]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = 0
    
    for rank in dup_ranks:
        hits_value += (rank <= k)
            
    return hits_value / len(dup_ranks)    

In [22]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = 0
    
    for rank in dup_ranks:
        dcg_value += (1 / math.log2(1 + rank)) * (rank <= k)
        
    return dcg_value / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [23]:
import pandas as pd

In [24]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


In [25]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [67]:
!unzip stackoverflow_similar_questions.zip

Считаем данные:

In [26]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(list(line.rstrip().split('\t')))
    return data

Нам понадобиться только файл validation.

In [27]:
validation_data = read_corpus('./stackoverflow_similar_questions/data/validation.tsv')

Кол-во строк

In [28]:
len(validation_data)

3760

Размер нескольких первых строк

In [29]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Функция ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        embeddings: наше векторное представление
        tokenizer: токенайзер
        dim: размер любого вектора в нашем представлении
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    zeros = 0
    temp_list = []
    q_vector = question_to_vec(question, embeddings, tokenizer)
    
    if q_vector.all() == 0:  # определим, является ли вектор вопроса нулевым
        zeros += 1
    
    for i, candidate in enumerate(candidates):
        c_vector = question_to_vec(candidate, embeddings, tokenizer)
        measure = cosine_similarity(q_vector.reshape(1, -1), c_vector.reshape(1, -1))
        temp_list.append((i, candidate, measure[0][0]))
    
    temp_list.sort(key=lambda x: x[2], reverse=True)  # отсортируем по косинусному расстоянию
    result = [(str(i), candidate) for i, candidate, measure in temp_list]  
    
    return zeros, result

Протестируем работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [32]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [33]:
for name, tokenizer in tokenizers.items():
    for question, q_candidates, i in zip(questions, candidates, range(2)):
        zeros, ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(f'Эксперимент {i+1}, {name}')
        print(f'Всего {zeros} нулевых векторов в вопросах')
        print(ranks)
        print()

Эксперимент 1, RE_tokenizer
Всего 0 нулевых векторов в вопросах
[('1', 'C# create cookie from string and send it'), ('0', 'Convert Google results object (pure js) to Python object'), ('2', 'How to use jQuery AJAX for an outside domain?')]

Эксперимент 2, RE_tokenizer
Всего 0 нулевых векторов в вопросах
[('0', 'Getting all list items of an unordered list in PHP'), ('2', 'select2 not displaying search results'), ('1', 'WPF- How to update the changes in list item of a list')]

Эксперимент 1, WP_tokenizer
Всего 0 нулевых векторов в вопросах
[('1', 'C# create cookie from string and send it'), ('0', 'Convert Google results object (pure js) to Python object'), ('2', 'How to use jQuery AJAX for an outside domain?')]

Эксперимент 2, WP_tokenizer
Всего 0 нулевых векторов в вопросах
[('0', 'Getting all list items of an unordered list in PHP'), ('2', 'select2 not displaying search results'), ('1', 'WPF- How to update the changes in list item of a list')]

Эксперимент 1, Spacy_tokenizer
Всего 0 нул

Все токенайзеры показали одинаковый результат.

Последовательность начальных индексов `для эксперимента 1`  1, 0, 2.

Последовательность начальных индексов `для эксперимента 2`  0, 2, 1.

Теперь мы можем оценить качество нашего метода:

In [34]:
from tqdm import tqdm

In [36]:
for name, tokenizer in tokenizers.items():
    sum_zero = 0
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        zeros, ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
        sum_zero += zeros
        wv_ranking.append([r[0] for r in ranks].index('0') + 1)
    print(name)
    print(f'Всего было {sum_zero} нулевых векторов в вопросах')
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    print()

 27%|██▋       | 1000/3760 [04:36<12:42,  3.62it/s]


RE_tokenizer
Всего было 0 нулевых векторов в вопросах
DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000



 27%|██▋       | 1000/3760 [04:35<12:41,  3.62it/s]


WP_tokenizer
Всего было 0 нулевых векторов в вопросах
DCG@   1: 0.410 | Hits@   1: 0.410
DCG@   5: 0.500 | Hits@   5: 0.580
DCG@  10: 0.521 | Hits@  10: 0.645
DCG@ 100: 0.568 | Hits@ 100: 0.875
DCG@ 500: 0.581 | Hits@ 500: 0.973
DCG@1000: 0.583 | Hits@1000: 1.000



 27%|██▋       | 1000/3760 [1:42:11<4:42:03,  6.13s/it]

Spacy_tokenizer
Всего было 3 нулевых векторов в вопросах
DCG@   1: 0.392 | Hits@   1: 0.392
DCG@   5: 0.482 | Hits@   5: 0.561
DCG@  10: 0.503 | Hits@  10: 0.627
DCG@ 100: 0.549 | Hits@ 100: 0.850
DCG@ 500: 0.565 | Hits@ 500: 0.967
DCG@1000: 0.568 | Hits@1000: 1.000



Наибольшую точность на `предобученных` эмбеддингах показал токенайзер `на основе регулярных выражений`. Чуть меньшую точность показал `WordPunkt` токенайзер, но в связи с тем, что в предобученных эмбеддингах очень мало знаков пунктуации, то разница от первого токенайзера минимальная. Неожиданно низкий результат показал токенайзер `Spacy` с лемматизацией. Не смотря на то, что в предобученных векторных представления присутствуют различные формы слова. Видимо в этой задаче нормализация не приносит пользу.

### Эмбеддинги, обученные на корпусе похожих вопросов

Улучшим качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim.

*В данной задаче размер окна будет подобран с помощью `GridSearch`, но обычно для коротких текстов берут `window_size` около `10` чтобы получить больше информации.*

In [35]:
train_data = read_corpus('./stackoverflow_similar_questions/data/train.tsv')

Количество строк:

In [36]:
len(train_data)

1000000

Выведем несколько строк датасета:

In [37]:
for i in range(5):
    print(train_data[i])
    print()

['converting string to list', 'Convert Google results object (pure js) to Python object']

['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?', 'Event handling for geometries in Three.js?']

['Sending array via Ajax fails', 'Getting all list items of an unordered list in PHP']

['How to insert CookieCollection to CookieContainer?', 'C# create cookie from string and send it']

['Updating one element of a bound Observable collection', 'WPF- How to update the changes in list item of a list']



#### Эмбеддинги без нормализации (лемматизации)

Подготовим препроцессинг данных для обучения эмбеддингов. Используем слова в нижнем регистре, со знаками пунктуации и без использования нормализации. Кроме того, отфильтруем слова согласно стоп-листа.

In [38]:
import nltk
import string
from nltk.tokenize import word_tokenize

stopWords = set(stopwords.words('english'))

def preproc_nltk(text):
#     text = re.sub('[^A-z]', ' ', text)
    return ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])

In [39]:
words = []
for questions in tqdm(train_data):
    question = ' '.join(q for q in questions)
    words.append(preproc_nltk(question).split())

100%|██████████| 1000000/1000000 [03:05<00:00, 5376.81it/s]


Выведем несколько строк подготовленного датасета:

In [40]:
for i in range(5):
    print(words[i])
    print()

['converting', 'string', 'list', 'convert', 'google', 'results', 'object', '(', 'pure', 'js', ')', 'python', 'object']

['html', '5', 'canvas', 'javascript', 'use', 'making', 'interactive', 'drawing', 'tool', '?', 'event', 'handling', 'geometries', 'three.js', '?']

['sending', 'array', 'via', 'ajax', 'fails', 'getting', 'list', 'items', 'unordered', 'list', 'php']

['insert', 'cookiecollection', 'cookiecontainer', '?', 'c', '#', 'create', 'cookie', 'string', 'send']

['updating', 'one', 'element', 'bound', 'observable', 'collection', 'wpf-', 'update', 'changes', 'list', 'item', 'list']



In [41]:
from gensim.models import Word2Vec

Подберём оптимальные параметры `min_count` и `windows` на примере `re_tokenizer` и алгоритме `CBOW`.

In [42]:
gs_1 = []
for count in tqdm(range(1, 11)):
    for window in range(1, 11):
        embeddings_trained = Word2Vec(words,    # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=count,  # consider words that occured at least 3 times
                              window=window,    # windows size
                              workers=8,        # num workers
                              sg=0).wv          # algorithm
        
        wv_ranking = []
        max_validation_examples = 1000
        for i, line in enumerate(validation_data):
            if i == max_validation_examples:
                break
            q, *ex = line
            _, ranks = rank_candidates(q, ex, embeddings_trained, re_tokenizer)
            wv_ranking.append([r[0] for r in ranks].index('0') + 1)
        for k in [1, 5, 10, 100, 500, 1000]:
            gs_1.append((count, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

100%|██████████| 10/10 [8:19:58<00:00, 2999.85s/it] 


In [43]:
def get_metrics(result):
    best_count, best_window, best_score = 0, 0, 0
    
    for res in result:
        num_count, num_window, k, dcg, hits = res
        if k==1 and hits > best_score:
            best_score = hits
            best_count = num_count
            best_window = num_window
    print(f"Лучший результат: Hits@1 = {best_score} при min_count = {best_count} и window = {best_window}") 

In [44]:
get_metrics(gs_1)

Лучший результат: Hits@1 = 0.427 при min_count = 7 и window = 10


Выполним аналогичный подбор параметров на втором алгоритме `skip-gram`.

In [46]:
gs_2 = []
for count in tqdm(range(1, 11)):
    for window in range(1, 11):
        embeddings_trained = Word2Vec(words,    # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=count,  # consider words that occured at least 3 times
                              window=window,    # windows size
                              workers=8,        # num workers
                              sg=1).wv          # algorithm
        
        wv_ranking = []
        max_validation_examples = 1000
        for i, line in enumerate(validation_data):
            if i == max_validation_examples:
                break
            q, *ex = line
            _, ranks = rank_candidates(q, ex, embeddings_trained, re_tokenizer)
            wv_ranking.append([r[0] for r in ranks].index('0') + 1)
        for k in [1, 5, 10, 100, 500, 1000]:
            gs_2.append((count, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

100%|██████████| 10/10 [10:28:14<00:00, 3769.50s/it] 


In [47]:
get_metrics(gs_2)

Лучший результат: Hits@1 = 0.539 при min_count = 5 и window = 10


Максимальная точность была достигнута при параметрах `min_count=5`, `window_size=10` и алгоритме `skip-gram`. Вероятно не стоит брать значение `min_count > 5` так как в этом случае в векторных представления будет значительно меньше слов. При `window_size > 10` окно по размерам будет приближаться к общей длине текста.

Используя полученные оптимальные параметры, проверим качество ранжирования для каждого из трёх токенайзеров:

In [50]:
embeddings_trained = Word2Vec(words,            # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=5,      # consider words that occured at least 3 times
                              window=10,        # windows size
                              workers=8,        # num workers
                              sg=1).wv          # algorithm

In [51]:
for name, tokenizer in tokenizers.items():
    sum_zero = 0
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        zeros, ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        sum_zero += zeros
        wv_ranking.append([r[0] for r in ranks].index('0') + 1)
    print(name)
    print(f'Всего было {sum_zero} нулевых векторов в вопросах')
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    print()

 27%|██▋       | 1000/3760 [04:25<12:13,  3.76it/s]


RE_tokenizer
Всего было 0 нулевых векторов в вопросах
DCG@   1: 0.536 | Hits@   1: 0.536
DCG@   5: 0.617 | Hits@   5: 0.687
DCG@  10: 0.640 | Hits@  10: 0.758
DCG@ 100: 0.674 | Hits@ 100: 0.914
DCG@ 500: 0.683 | Hits@ 500: 0.989
DCG@1000: 0.684 | Hits@1000: 1.000



 27%|██▋       | 1000/3760 [04:28<12:22,  3.72it/s]


WP_tokenizer
Всего было 0 нулевых векторов в вопросах
DCG@   1: 0.517 | Hits@   1: 0.517
DCG@   5: 0.601 | Hits@   5: 0.676
DCG@  10: 0.621 | Hits@  10: 0.737
DCG@ 100: 0.656 | Hits@ 100: 0.902
DCG@ 500: 0.666 | Hits@ 500: 0.979
DCG@1000: 0.668 | Hits@1000: 1.000



 27%|██▋       | 1000/3760 [1:34:05<4:19:42,  5.65s/it]

Spacy_tokenizer
Всего было 1 нулевых векторов в вопросах
DCG@   1: 0.510 | Hits@   1: 0.510
DCG@   5: 0.593 | Hits@   5: 0.665
DCG@  10: 0.615 | Hits@  10: 0.734
DCG@ 100: 0.649 | Hits@ 100: 0.893
DCG@ 500: 0.659 | Hits@ 500: 0.974
DCG@1000: 0.662 | Hits@1000: 1.000



#### Эмбеддинги с нормализацией (лемматизацией)

Подготовим препроцессинг данных для обучения эмбеддингов c лемматизацией. Также используем слова в нижнем регистре, со знаками пунктуации, отфильтруем слова согласно стоп-листа.

In [52]:
def preproc_nltk_lemma(text):
    text = ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])
    return spacy_tokenizer.tokenize(text)

In [53]:
words_lemma = []
for questions in tqdm(train_data):
    question = ' '.join(q for q in questions)
    words_lemma.append(preproc_nltk_lemma(question))

100%|██████████| 1000000/1000000 [1:51:09<00:00, 149.93it/s]


Выведем несколько строк подготовленного датасета:

In [54]:
for i in range(5):
    print(words_lemma[i])
    print()

['convert', 'string', 'list', 'convert', 'google', 'result', 'object', '(', 'pure', 'js', ')', 'python', 'object']

['html', '5', 'canvas', 'javascript', 'use', 'make', 'interactive', 'drawing', 'tool', '?', 'event', 'handling', 'geometry', 'three.js', '?']

['send', 'array', 'via', 'ajax', 'fail', 'get', 'list', 'item', 'unordered', 'list', 'php']

['insert', 'cookiecollection', 'cookiecontainer', '?', 'c', '#', 'create', 'cookie', 'string', 'send']

['update', 'one', 'element', 'bind', 'observable', 'collection', 'wpf-', 'update', 'change', 'list', 'item', 'list']



Обучим эмбеддинги с нормализацией на полученных ранее оптимальных параметрах:

In [55]:
embeddings_trained_lemma = Word2Vec(words_lemma,  # data for model to train on
                           vector_size=200,       # embedding vector size
                           min_count=5,           # consider words that occured at least 3 times
                           window=10,             # windows size
                           workers=8,             # num workers
                           sg=1).wv               # algorithm

Проверим качество ранжирования для каждого из трёх токенайзеров:

In [56]:
for name, tokenizer in tokenizers.items():
    sum_zero = 0
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        zeros, ranks = rank_candidates(q, ex, embeddings_trained_lemma, tokenizer)
        sum_zero += zeros
        wv_ranking.append([r[0] for r in ranks].index('0') + 1)
    print(name)
    print(f'Всего было {sum_zero} нулевых векторов в вопросах')
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    print()

 27%|██▋       | 1000/3760 [04:37<12:44,  3.61it/s]


RE_tokenizer
Всего было 1 нулевых векторов в вопросах
DCG@   1: 0.359 | Hits@   1: 0.359
DCG@   5: 0.438 | Hits@   5: 0.504
DCG@  10: 0.457 | Hits@  10: 0.565
DCG@ 100: 0.499 | Hits@ 100: 0.771
DCG@ 500: 0.520 | Hits@ 500: 0.938
DCG@1000: 0.527 | Hits@1000: 1.000



 27%|██▋       | 1000/3760 [04:37<12:46,  3.60it/s]


WP_tokenizer
Всего было 1 нулевых векторов в вопросах
DCG@   1: 0.349 | Hits@   1: 0.349
DCG@   5: 0.425 | Hits@   5: 0.491
DCG@  10: 0.446 | Hits@  10: 0.555
DCG@ 100: 0.486 | Hits@ 100: 0.754
DCG@ 500: 0.509 | Hits@ 500: 0.936
DCG@1000: 0.516 | Hits@1000: 1.000



 27%|██▋       | 1000/3760 [1:39:44<4:35:17,  5.98s/it]

Spacy_tokenizer
Всего было 0 нулевых векторов в вопросах
DCG@   1: 0.378 | Hits@   1: 0.378
DCG@   5: 0.457 | Hits@   5: 0.526
DCG@  10: 0.476 | Hits@  10: 0.586
DCG@ 100: 0.515 | Hits@ 100: 0.782
DCG@ 500: 0.536 | Hits@ 500: 0.943
DCG@1000: 0.542 | Hits@1000: 1.000



## Вывод:

Во всех проведённых экспериментах наилучшее качество показал токенайзер на `основе регулярных выражений` (`RE tokenizer`), чуть хуже показал себя `WordPunkt` токенайзер с токенизацией знаков пунктуации и на последнем месте `Spacy`, который дополнительно проводит нормализацию (лемматизацию) слов. Такие результаты я связываю с особенностями исходного корпуса. Для вопросов и ответов на StackOverFlow не так важны знаки препинания (это не художественный текст) и формы слов, сколько семантика. Данный тип текстов скорее технические диалоги в виде свободной формы общения, которые могут позволять себе некоторые вольности. Поэтому обычный токенайзер без учёта пунктуации и лемматизации показал в данной задаче наилучшее качество. Конечно, на другом корпусе (например на корпусе текстов художественной литературы) результат может быть полностью противоположный. Кроме того, токенизация с лемматизацией позволяет снизить размер словаря, но в данной задаче это не помогло увеличить качество.

С учётом вышесказанного, эмбеддинги без нормализации показали себя лучше чем эмбеддинги созданные с предварительной нормализацией (лемматизацией) слов. Если сравнить предобученные эмбеддинги с обученными, то последние показали точность выше как на `cbow` так и, особенно, на `skip-gram` не смотря почти на вдвое меньший объём векторного пространства. Я связываю это с подбором оптимальных параметров `window` и `min_count` для текущнй задачи, что позволило увеличить итоговую точность.

Итоговый результат получился не очень впечатляющий (максимальная точность `DCG@1: 0.536`, `Hits@1: 0.536`). Вероятно это связано с тем, что `Word2Vec` подход достаточно устаревший и имеет ряд существенных недостатков, таких как - отсутствие информации о предложении или контексте, в котором используется слово; не учитывается совместная встречаемость слов и разное значение одного и того же слова в зависимости от контекста. В качестве улучшения можно использовать, например, модель `GloVe` (Global Vectors), которая учитывает частоту встречаемость слов и опережает Word2Vec в большинстве тестов. Или библиотеку `fastText`, которая к основной модели Word2Vec добавляет модель символьных n-грамм и способна генерировать эмбеддинги и для неизвестных слов. Это не говоря уже о таких методах как реккурентные нейронные сети, трансформеры, языковые модели GPT, BERT.